In [1]:
!pip uninstall -y numpy
!pip install numpy==1.24.4

Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4
  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.3 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is in

In [4]:
!pip install --force-reinstall --no-cache-dir torch torchvision opencv-python ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 109.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 281.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 232.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 174.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 293.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install ultralytics==8.2.103 -q

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.11.12 torch-2.7.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 44.7/112.6 GB disk)


In [17]:
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.colors as mcolors

# Define known classes
class_id_to_name = {
    0: 'pallet',
    1: 'stillage',
    3: 'small_load_carrier'
}

# Define colors for known classes
category_colors = {
    0: 'red',
    1: 'blue',
    3: 'orange'
}

def draw_boxes_on_frame(frame, boxes, labels, box_thickness=4):
    for box, label in zip(boxes, labels):
        if label not in class_id_to_name:
            continue  # Skip unknown classes
        x1, y1, x2, y2 = map(int, box)
        color = category_colors[label]
        if isinstance(color, str):
            color = mcolors.to_rgb(color)
        # Convert RGB to BGR for OpenCV
        color_bgr = tuple(int(c * 255) for c in reversed(color))

        # Draw rectangle (bounding box)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color_bgr, thickness=box_thickness)

        # Put class name text above bounding box
        class_name = class_id_to_name[label]
        font_scale = 1.0
        thickness = 2
        (text_w, text_h), _ = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)

        # Draw filled rectangle behind text for better visibility
        cv2.rectangle(frame, (x1, y1 - text_h - 10), (x1 + text_w, y1), color_bgr, -1)

        # Draw text (class name)
        cv2.putText(frame, class_name, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)
    return frame

# Load YOLO model
model_path = '/content/best.pt'
model = YOLO(model_path)

# Open video
video_path = '/content/worker-in-warehouse-riding-on-forklift-trucks-with-cardboard-boxes-man-wearing-SBV-316451680-preview.mp4'  # Remplace par ton chemin vidéo
cap = cv2.VideoCapture(video_path)

# Prepare output video writer
output_path = 'output_annotated_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, conf=0.25, save=False)
    result = results[0]

    boxes = result.boxes.xyxy.cpu().numpy()
    labels = result.boxes.cls.cpu().numpy().astype(int)

    # Filter boxes and labels only for known classes
    valid_indices = [i for i, lbl in enumerate(labels) if lbl in class_id_to_name]
    boxes = [boxes[i] for i in valid_indices]
    labels = [labels[i] for i in valid_indices]

    # Draw bounding boxes with class names
    frame = draw_boxes_on_frame(frame, boxes, labels, box_thickness=4)

    # No legend bar

    # Write frame to output video
    out.write(frame)

    frame_count += 1
    print(f"Processed frame {frame_count}")

cap.release()
out.release()
print(f"Saved annotated video to {output_path}")



0: 256x416 37 pallets, 11.5ms
Speed: 1.6ms preprocess, 11.5ms inference, 1.4ms postprocess per image at shape (1, 3, 256, 416)
Processed frame 1

0: 256x416 35 pallets, 9.8ms
Speed: 2.1ms preprocess, 9.8ms inference, 1.3ms postprocess per image at shape (1, 3, 256, 416)
Processed frame 2

0: 256x416 36 pallets, 9.4ms
Speed: 1.5ms preprocess, 9.4ms inference, 1.3ms postprocess per image at shape (1, 3, 256, 416)
Processed frame 3

0: 256x416 36 pallets, 9.1ms
Speed: 1.5ms preprocess, 9.1ms inference, 1.2ms postprocess per image at shape (1, 3, 256, 416)
Processed frame 4

0: 256x416 38 pallets, 9.4ms
Speed: 1.4ms preprocess, 9.4ms inference, 1.2ms postprocess per image at shape (1, 3, 256, 416)
Processed frame 5

0: 256x416 38 pallets, 8.8ms
Speed: 1.4ms preprocess, 8.8ms inference, 1.3ms postprocess per image at shape (1, 3, 256, 416)
Processed frame 6

0: 256x416 37 pallets, 9.8ms
Speed: 1.4ms preprocess, 9.8ms inference, 1.3ms postprocess per image at shape (1, 3, 256, 416)
Processe